<a href="https://colab.research.google.com/github/AsadiAhmad/Face-Recognition/blob/main/Code/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import Libraries

In [25]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import gdown

# Step 2: Download Resources

In [26]:
gdown.download(id="1IvTKQkfYD8UOMjGPAQWY0iQK8--Qnenv", output="face_detection_yunet_2023mar.onnx", quiet=False)
gdown.download(id="1Sg-y47n8Z5K099ytdgpIkVw2lOk8kD9n", output="face_recognition_sface_2021dec.onnx", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1IvTKQkfYD8UOMjGPAQWY0iQK8--Qnenv
To: /content/face_detection_yunet_2023mar.onnx
100%|██████████| 233k/233k [00:00<00:00, 4.94MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Sg-y47n8Z5K099ytdgpIkVw2lOk8kD9n
To: /content/face_recognition_sface_2021dec.onnx
100%|██████████| 38.7M/38.7M [00:00<00:00, 126MB/s]


'face_recognition_sface_2021dec.onnx'

In [27]:
!wget https://raw.githubusercontent.com/AsadiAhmad/Face-Recognition/main/Pictures/elon_musk.jpg -O elon_musk.jpg
!wget https://raw.githubusercontent.com/AsadiAhmad/Face-Recognition/main/Videos/conference.mp4 -O conference.mp4

--2025-05-12 12:25:57--  https://raw.githubusercontent.com/AsadiAhmad/Face-Recognition/main/Pictures/elon_musk.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31340 (31K) [image/jpeg]
Saving to: ‘elon_musk.jpg’

elon_musk.jpg       100%[===================>]  30.61K  --.-KB/s    in 0.02s   

2025-05-12 12:25:57 (1.94 MB/s) - ‘elon_musk.jpg’ saved [31340/31340]

--2025-05-12 12:25:57--  https://raw.githubusercontent.com/AsadiAhmad/Face-Recognition/main/Videos/conference.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337380

# Step 3: Load Images and Videos

In [28]:
image = cv.imread('elon_musk.jpg')
cap = cv.VideoCapture('conference.mp4')

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Step 4: Define Detector Model

In [29]:
detector = cv.FaceDetectorYN.create(
    "face_detection_yunet_2023mar.onnx",
    "",
    (320, 320),
    0.8,
    0.3,
    5000
)

# Step 5: Define Recognizer Model

In [30]:
recognizer = cv.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx","")

# Step 6: Extracting feature for the selected photo

In [31]:
height, width, _ = image.shape
detector.setInputSize((width, height))
main_faces = detector.detect(image)
main_face_align = recognizer.alignCrop(image, main_faces[1][0])
main_face_feature = recognizer.feature(main_face_align)

# Step 7: Set Parameters

In [32]:
fps = int(cap.get(cv.CAP_PROP_FPS))
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

output_path = 'face_detected.mp4'
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter(output_path, fourcc, fps, (width, height))

detector.setInputSize((width, height))

# Step 8: Process Video Frame by Frame

In [33]:
def rectangle_faces(input, faces, index, color=(0, 0, 255), thickness=2):
    if faces[1] is not None:
        for idx, face in enumerate(faces[1]):
            coords = face[:-1].astype(np.int32)
            if idx == index:
                cv.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), thickness)
            else:
                cv.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), color, thickness)

In [34]:
def match_faces(face_feature1, face_feature2):
    cosine_similarity_threshold = 0.363
    cosine_score = recognizer.match(face_feature1, face_feature2, cv.FaceRecognizerSF_FR_COSINE)

    if cosine_score >= cosine_similarity_threshold:
        return True
    else:
        return False

In [35]:
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Reached end of video.")
        break

    detected_face_index = -1
    faces = detector.detect(frame)

    if faces[1] is not None:
        for index, face in enumerate(faces[1]):
            face_align = recognizer.alignCrop(frame, face)
            face_feature = recognizer.feature(face_align)

            if match_faces(face_feature, main_face_feature) and detected_face_index == -1:
                detected_face_index = index

        rectangle_faces(frame, faces, detected_face_index)
    out.write(frame)

cap.release()
out.release()

Reached end of video.
